In [ ]:
def plt_imshow(title='image', img=None, figsize=(8 ,5)):
    plt.figure(figsize=figsize)
 
    if type(img) == list:
        if type(title) == list:
            titles = title
        else:
            titles = []
 
            for i in range(len(img)):
                titles.append(title)
 
        for i in range(len(img)):
            if len(img[i].shape) <= 2:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_GRAY2RGB)
            else:
                rgbImg = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
 
            plt.subplot(1, len(img), i + 1), plt.imshow(rgbImg)
            plt.title(titles[i])
            plt.xticks([]), plt.yticks([])
 
        plt.show()
    else:
        if len(img.shape) < 3:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        else:
            rgbImg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 
        plt.imshow(rgbImg)
        plt.title(title)
        plt.xticks([]), plt.yticks([])
        plt.show()
 
 
def make_scan_image(image, width, ksize=(5,5), min_threshold=75, max_threshold=200):
  image_list_title = []
  image_list = []
 
  image = imutils.resize(image, width=width)
  ratio = org_image.shape[1] / float(image.shape[1])
 
  # 이미지를 grayscale로 변환하고 blur를 적용
  # 모서리를 찾기위한 이미지 연산
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blurred = cv2.GaussianBlur(gray, ksize, 0)
  edged = cv2.Canny(blurred, min_threshold, max_threshold)
 
  image_list_title = ['gray', 'blurred', 'edged']
  image_list = [gray, blurred, edged]
 
  # contours를 찾아 크기순으로 정렬
  cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
 
  findCnt = None
 
  # 정렬된 contours를 반복문으로 수행하며 4개의 꼭지점을 갖는 도형을 검출
  for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    

    # contours가 크기순으로 정렬되어 있기때문에 제일 첫번째 사각형을 영역으로 판단하고 break
    if len(approx) == 4:
      findCnt = approx
      break
 
 
  # 만약 추출한 윤곽이 없을 경우 오류
  if findCnt is None:
    raise Exception(("Could not find outline."))
 
 
  output = image.copy()
  cv2.drawContours(output, [findCnt], -1, (0, 255, 0), 2)
  
  image_list_title.append("Outline")
  image_list.append(output)
 
  # 원본 이미지에 찾은 윤곽을 기준으로 이미지를 보정
  transform_image = four_point_transform(org_image, findCnt.reshape(4, 2) * ratio)
 
  plt_imshow(image_list_title, image_list)
  plt_imshow("Transform", transform_image)
 
  return transform_image

# OpenCV putText의 경우 한글지원이 되지 않기 때문에 한글을 표기하기 위한 Function을 만들어 사용


def putText(cv_img, text, x, y, color=(0, 0, 0), font_size=30):
  font = ImageFont.truetype("fonts/malgunbd.ttf", font_size)
  img = Image.fromarray(cv_img)
   
  draw = ImageDraw.Draw(img)
  draw.text((x, y), text, font=font, fill=color)
 
  cv_img = np.array(img)
  
  return cv_img




from PIL import Image
from PIL import ImageDraw
import numpy as np
import matplotlib.pyplot as plt

def pil_draw_rect(image, point1, point2):
    draw = ImageDraw.Draw(image)
    draw.rectangle((point1, point2), outline=(0, 0, 0), width=5)

    return image



a = []
ame = os.listdir('C:/Users/smhrd/yolov5test/result/')
for i in range(0,len(ame)) :
   
        image = Image.open('C:/Users/smhrd/yolov5test/result/'+ame[i])
        plt.xticks(color='w')
        plt.yticks(color='w')
        x= image.size[0]
        y= image.size[1]
        image = pil_draw_rect(image, (0, 0), (x, y))

        plt.imshow(np.array(image))
        plt.savefig('test/'+ame[i])
        img = load_img('test/'+ame[i])
        org_image = np.asarray(img,dtype=np.uint8)
        #plt_imshow("orignal image", org_image )
        business_card_image = make_scan_image(org_image, width=200, ksize=(5, 5), min_threshold=20, max_threshold=100)


        langs = ['ko', 'en']

        reader = Reader(lang_list=langs, gpu=True)
        results = reader.readtext(business_card_image)


        simple_results = reader.readtext(business_card_image, detail = 0)
        print(simple_results)
        a.append(simple_results)
        A8 =["함기","강묵","시율","서울","서율","피루","부산","무산","대구","인천","인진","광주","대전","울산","미전"]
        A9 =["서울","피루","서율","부산","무산","대구","인천","인진","광주","대전","울산","세종","경기","강원","충북","중북","충남","중남","전북","전남","진남","미전"
          "경북","경남","제주","경개","감기","경묵"]
        A10 = ["피루","서울","서율","부산","무산","인천","인진","경기","제주"]
        B = ["굉주","연수","수영","님","꼼정","무껑","부멍","내무","무망","미주올","장원","기장","다주",'삼즈',"증로","종로","중","용산","성동","광진","동대문","중랑","성북","강북","도봉","노원","은평","서대문","마포","양천","강서","구로","금천","영등포","동작","관악","서초","강남","송파","강동","중","서","동","영동","부산진","동래","남","북","해운대","사하","금정","강서","연제","수영","사상","기장","동","서","남","북","중","수성","달서","달성","중","동","미추홀","연수","남동","부평","계양","서","강화","옹진","광산","유성","대덕","울주","세종","고양","수원","용인","과천","광명","광주","구리","군포","김포","남양주","동두천","부천","성남","시흥","안산","안성","안양","양주","여주","오산","의왕","의정부","이천","파주","평택","포천","하남","화성","가평","양평",",연천","개성","개풍","장단","강릉","동해","삼척","속초","원주","춘천","태백","고성","양구","양양","영월","인제","정선","철원","평창","홍천","화천","횡성","김화","이천","통천","평강","회양","제천","청주","충주","괴산","단양","보은","영동","옥천","음성","증평","진천","계룡","공주","논산","당진","보령","서산","아산","천안","금산","부여","서천","예산","청양","태안","홍성","군산","김제","남원","익산","전주","정읍","고창","무주","부안","순창","완주","임실","장수","진안","광양","나주","목포","순천","여수","강진","고흥","곡성","구례","담양","무안","보성","신안","영광","영암","완도","장성","장흥","진도","함평","해남","화순","경산","경주","구미","김천","문경","상주","안동","영주","영천","포항","고령","군위","봉화","성주","영덕","영양","예천","울릉","울진","의성","청도","청송","칠곡","창원","거제","김해","밀양","사천","양산","진주","통영","거창","고성","남해","산청","의령","창녕","하동","함안","함양","합천","제주","서귀포"]


        if len(simple_results) == 3 :
            a = simple_results[0].split() 
            a.append(simple_results[1])
            a.append(simple_results[2])
        elif len(simple_results) == 2 :
            a = simple_results[0].split() 
            a.append(simple_results[1][0])
            a.append(simple_results[1][1:5])


        if(a[0] in A10) and (a[1] in B) and (len(a[2])+len(a[3]) == 5) and  (len(a[0]) + len(a[1])+ len(a[2])+ len(a[3]))==10:
            print("정상")

        elif(a[0] in A9) and (a[1] in B) and (len(a[2])+len(a[3]) == 5) and len(a[0]) + len(a[1])+ len(a[2])+ len(a[3])==9:
            print("정상")
        elif(a[0] in A8) and (a[1] in B) and (len(a[2])+len(a[3]) == 5) and len(a[0]) + len(a[1])+ len(a[2])+ len(a[3])==8:
            print("정상")
        else :
            print("비정상")


#         import cx_Oracle
#         dsn = cx_Oracle.makedsn("project-db-stu.ddns.net",1524,service_name="XE")

#         connection = cx_Oracle.connect(user="bang",password="1234",dsn=dsn, encoding="UTF-8")

#      #   cur = connection.cursor()
#      #   sql = "update T_REPORT set damage=:1 where IMAGE_FILE=:2"
#      #   re2 = "FB_IMG_1615524547046.jpg"
#      #   cur.execute(sql, (re1, re2))
#     #  connection.commit()
#         print(re1)